In [1]:
!pwd

/home/student/surg-pose-sim2real/src


In [2]:
!ls /datashare/project

camera.json  surgical_tools_models  train2017.zip  vids_tune
haven	     train2017		    vids_test


In [3]:
!ls /datashare/project/train2017

000000000009.jpg  000000143170.jpg  000000287519.jpg  000000433097.jpg
000000000034.jpg  000000143182.jpg  000000287528.jpg  000000433116.jpg
000000000049.jpg  000000143217.jpg  000000287531.jpg  000000433123.jpg
000000000071.jpg  000000143221.jpg  000000287565.jpg  000000433133.jpg
000000000078.jpg  000000143236.jpg  000000287567.jpg  000000433151.jpg
000000000081.jpg  000000143252.jpg  000000287574.jpg  000000433156.jpg
000000000089.jpg  000000143275.jpg  000000287575.jpg  000000433170.jpg
000000000208.jpg  000000143286.jpg  000000287636.jpg  000000433184.jpg
000000000241.jpg  000000143306.jpg  000000287639.jpg  000000433197.jpg
000000000247.jpg  000000143320.jpg  000000287651.jpg  000000433198.jpg
000000000283.jpg  000000143323.jpg  000000287695.jpg  000000433202.jpg
000000000315.jpg  000000143331.jpg  000000287723.jpg  000000433214.jpg
000000000321.jpg  000000143334.jpg  000000287737.jpg  000000433235.jpg
000000000338.jpg  000000143339.jpg  000000287748.jpg  000000433297.jpg
000000

In [4]:
!ls /datashare/project/haven

hdris


## The Available Data :

In [7]:
print("Surgical Tools Models:")
!ls /datashare/project/surgical_tools_models  
print("---------------------------------------------------------------------")
print("Differents Videos:")
!ls /datashare/project/vids_tune
print("---------------------------------------------------------------------")
print("Background Images:")
!ls /datashare/project/haven/hdris
print("---------------------------------------------------------------------")
print("Differents Videos for Test:")
!ls /datashare/project/vids_test

Surgical Tools Models:
needle_holder  tweezers
---------------------------------------------------------------------
Differents Videos:
20_2_24_1.mp4  20_2_24_1_small.mp4  4_2_24_B_2.mp4  4_2_24_B_2_small.mp4
---------------------------------------------------------------------
Background Images:
abandoned_bakery		       mealie_road
abandoned_church		       medieval_cafe
abandoned_construction		       metro_noord
abandoned_factory_canteen_01	       metro_vijzelgracht
abandoned_factory_canteen_02	       missile_launch_facility_01
abandoned_games_room_01		       missile_launch_facility_02
abandoned_games_room_02		       missile_launch_facility_03
abandoned_greenhouse		       misty_dawn
abandoned_hall_01		       misty_farm_road
abandoned_hopper_terminal_01	       misty_pines
abandoned_hopper_terminal_02	       modern_bathroom
abandoned_hopper_terminal_03	       modern_buildings
abandoned_hopper_terminal_04	       modern_buildings_2
abandoned_parking		       modern_buildings_night
abandone

In [8]:
print("Background Images file format:")
!ls /datashare/project/haven/hdris/urban_alley_01			       			       			       

Background Images file format:
urban_alley_01_2k.hdr


In [9]:
print("Inspect Surgical Tool - needle_holder:")
!ls /datashare/project/surgical_tools_models/needle_holder  
print("---------------------------------------------------------------------")
print("Inspect Surgical Tool - tweezers:")
!ls /datashare/project/surgical_tools_models/tweezers  

Inspect Surgical Tool - needle_holder:
NH1.mtl   NH11.mtl  NH13.mtl  NH15.mtl	NH3.mtl  NH5.mtl  NH7.mtl  NH9.mtl
NH1.obj   NH11.obj  NH13.obj  NH15.obj	NH3.obj  NH5.obj  NH7.obj  NH9.obj
NH10.mtl  NH12.mtl  NH14.mtl  NH2.mtl	NH4.mtl  NH6.mtl  NH8.mtl
NH10.obj  NH12.obj  NH14.obj  NH2.obj	NH4.obj  NH6.obj  NH8.obj
---------------------------------------------------------------------
Inspect Surgical Tool - tweezers:
T1.mtl	T10.mtl  T2.mtl  T3.mtl  T4.mtl  T5.mtl  T6.mtl  T7.mtl  T8.mtl  T9.mtl
T1.obj	T10.obj  T2.obj  T3.obj  T4.obj  T5.obj  T6.obj  T7.obj  T8.obj  T9.obj


In [16]:
!ls -lh /home/student/surg-pose-sim2real/src/synthetic_data_generator.py


-rw-rw-r-- 1 student student 1.3K Aug 24 15:32 /home/student/surg-pose-sim2real/src/synthetic_data_generator.py


In [4]:
import os, json, numpy as np
from pathlib import Path

NEEDLE_SRC   = Path("/datashare/project/surgical_tools_models/needle_holder")
TWEEZERS_SRC = Path("/datashare/project/surgical_tools_models/tweezers")

KPS_ROOT = Path("/home/student/surg-pose-sim2real/assets/kps")  # writable
( KPS_ROOT / "needle_holder" ).mkdir(parents=True, exist_ok=True)
( KPS_ROOT / "tweezers"     ).mkdir(parents=True, exist_ok=True)

SCHEMA = ["tip","jaw_L","jaw_R","hinge_base","handle_end"]

def read_obj_vertices(p: Path):
    V=[]
    with open(p, "r", errors="ignore") as f:
        for line in f:
            if line.startswith("v "):
                _, xs, ys, zs, *rest = line.strip().split()
                V.append([float(xs), float(ys), float(zs)])
    V=np.array(V, dtype=float)
    if V.size==0: raise ValueError(f"No vertices in {p}")
    return V

def pca_axes(V):
    C=V.mean(axis=0); X=V-C
    _,_,VT=np.linalg.svd(X, full_matrices=False)
    U=VT
    if np.linalg.det(U)<0: U[2]*=-1
    return C,U

def landmarks_from_pca(V):
    C,U = pca_axes(V)
    X = (V-C)@U.T
    t,y,z = X[:,0], X[:,1], X[:,2]
    tip_idx     = np.argmax(t)
    handle_idx  = np.argmin(t)
    thr=np.percentile(t,90); near=X[t>=thr]; near_idx=np.where(t>=thr)[0]
    if len(near)>=2:
        jawL_idx = near_idx[np.argmax(near[:,1])]
        jawR_idx = near_idx[np.argmin(near[:,1])]
    else:
        jawL_idx = np.argmax(y); jawR_idx = np.argmin(y)
    # hinge ~ narrowest cross-section away from ends
    lo,hi=np.percentile(t,15),np.percentile(t,85)
    mask=(t>=lo)&(t<=hi); tsel,yse,zse = t[mask], y[mask], z[mask]
    if len(tsel)>=20:
        bins=40; edges=np.linspace(lo,hi,bins+1)
        radii=[]; centers=[]
        for b in range(bins):
            m=(tsel>=edges[b])&(tsel<edges[b+1])
            if m.sum()>=10:
                radii.append(np.sqrt(np.mean(yse[m]**2+zse[m]**2)))
                centers.append(0.5*(edges[b]+edges[b+1]))
        hinge_idx = np.argmin(np.abs(t - centers[int(np.argmin(radii))])) if radii else np.argmin(np.abs(t-np.median(t)))
    else:
        hinge_idx = np.argmin(np.abs(t-np.median(t)))
    return {
        "tip":        V[tip_idx],
        "jaw_L":      V[jawL_idx],
        "jaw_R":      V[jawR_idx],
        "hinge_base": V[hinge_idx],
        "handle_end": V[handle_idx]
    }

def write_sidecar(dst_dir: Path, obj_path: Path, pts: dict):
    out = {"space":"local","points":{k:[float(v[0]),float(v[1]),float(v[2])] for k,v in pts.items()}}
    jp = dst_dir / (obj_path.stem + "_kps.json")
    with open(jp,"w") as f: json.dump(out,f,indent=2)
    return jp

def process(src_dir: Path, dst_subdir: str):
    dst = KPS_ROOT / dst_subdir
    written=[]
    for obj in sorted(src_dir.glob("*.obj")):
        try:
            V = read_obj_vertices(obj)
            pts = landmarks_from_pca(V)
            jp = write_sidecar(dst, obj, pts)
            written.append(jp)
        except Exception as e:
            print("[warn]", obj, "->", e)
    return written

w1 = process(NEEDLE_SRC, "needle_holder")
w2 = process(TWEEZERS_SRC, "tweezers")
print("Sidecars written:", len(w1)+len(w2))
print("Example:", (w1+w2)[:3])

Sidecars written: 25
Example: [PosixPath('/home/student/surg-pose-sim2real/assets/kps/needle_holder/NH1_kps.json'), PosixPath('/home/student/surg-pose-sim2real/assets/kps/needle_holder/NH10_kps.json'), PosixPath('/home/student/surg-pose-sim2real/assets/kps/needle_holder/NH11_kps.json')]


### Picking Ratios

In [12]:
from pathlib import Path
import random
# HDRIs: accept *_2k.hdr / *.hdr / *.exr inside each subdir
HDRI_LIST = []
hdr_root = Path("/datashare/project/haven/hdris")
if hdr_root.exists():
    for sub in hdr_root.iterdir():
        if sub.is_dir():
            cands = list(sub.glob("*_2k.hdr")) + list(sub.glob("*.hdr")) + list(sub.glob("*.exr"))
            HDRI_LIST.extend(str(p) for p in cands)

random.shuffle(HDRI_LIST)
print(f"[assets] HDRIs: {len(HDRI_LIST)}")
HDRI_COUNT = len(HDRI_LIST)

[assets] HDRIs: 1472


In [13]:
# Backgrounds (2D) for compositing
BG_LIST = sorted([p for p in Path("/datashare/project/train2017").glob("*.*") if p.suffix.lower() in (".jpg",".jpeg",".png")])
print(f"[assets] 2D backgrounds: {len(BG_LIST)}")
BG_COUNT = len(BG_LIST)

[assets] 2D backgrounds: 32612


In [18]:
#studio=0.5,hdri=0.2,composite=0.3
TOTAL = 1.0
STUDIO_NUM = 0.1
HDRI_NUM = 10 * float(float(HDRI_COUNT) / float(HDRI_COUNT + BG_COUNT)) * (TOTAL - STUDIO_NUM)
BG_NUM = TOTAL - STUDIO_NUM - HDRI_NUM


In [22]:
print("studio_ratio:", STUDIO_NUM)
print("hdri_ratio:", HDRI_NUM)
print("composite_ratio:", BG_NUM)

studio_ratio: 0.1
hdri_ratio: 0.38868677385283423
composite_ratio: 0.5113132261471658


In [ ]:
!OUT=/home/student/surg-pose-sim2real/outputs/synthetic_images_data; \
MPLBACKEND=Agg python -m blenderproc run /home/student/surg-pose-sim2real/src/synthetic_data_generator.py \
  --outdir "$OUT" --max_images 1150 --per_obj_cap 50 \
  --mix 'studio=0.1','hdri=0.38868677385283423','composite=0.5113132261471658' \
  --samples 24 --res_scale 0.8 --device gpu \
  --kps_dir /home/student/surg-pose-sim2real/assets/kps

Using blender in /home/student/blender/blender-4.2.1-linux-x64
Using temporary directory: /dev/shm/blender_proc_e6464d448cd8452ab058da5039a66f7b
Blender 4.2.1 LTS (hash 396f546c9d82 built 2024-08-19 23:32:23)
Selecting render devices...
Device Tesla V100-PCIE-16GB of type OPTIX found and used.
Device Intel Xeon CPU E5-2690 v4 @ 2.60GHz of type CPU found and used.
[init] Cycles device = GPU (OPTIX/CUDA)
[assets] needle_holder=15 | tweezers=10


## Phase 2: 2D Pose Estimation Development for Surgical Instruments

In [ ]:
# 0) Paths
DATA_YAML = "/home/student/surg-pose-sim2real/outputs/datasets/surg.yaml"   
# 1) Small baseline you already did (optional to reproduce)
!python -m src.train --model yolov8n-pose.pt --data {DATA_YAML} \
  --imgsz 896 --epochs 150 --batch 32 --project runs_pose --name synth_v8n_896

# 2) v8s-pose (primary)
!python -m src.train --model yolov8s-pose.pt --data {DATA_YAML} \
  --imgsz 896 --epochs 200 --batch 24 --optimizer AdamW --lr0 0.001 --lrf 0.01 \
  --project runs_pose --name synth_v8s_896

# 3) v8m-pose (if VRAM ≥ 16GB; otherwise drop batch)
!python -m src.train --model yolov8m-pose.pt --data {DATA_YAML} \
  --imgsz 960 --epochs 200 --batch 16 --optimizer AdamW --lr0 0.001 --lrf 0.01 \
  --project runs_pose --name synth_v8m_960



In [1]:
import pandas as pd, yaml, pathlib as P

def summarize_runs(run_dirs):
    rows = []
    for rd in map(P.Path, run_dirs):
        csv = rd/'results.csv'
        args = rd/'args.yaml'
        best = rd/'weights'/'best.pt'
        if not csv.exists(): continue
        df = pd.read_csv(csv)
        last = df.iloc[-1].to_dict()
        cfg = yaml.safe_load(args.read_text()) if args.exists() else {}
        rows.append({
            'run': rd.name,
            'model': cfg.get('model', '?'),
            'imgsz': cfg.get('imgsz', '?'),
            'epochs': cfg.get('epochs', '?'),
            'mAP50-95(P)': last.get('metrics/mAP50-95(P)', float('nan')),
            'mAP50(P)': last.get('metrics/mAP50(P)', float('nan')),
            'P(P)': last.get('metrics/precision(P)', float('nan')),
            'R(P)': last.get('metrics/recall(P)', float('nan')),
            'mAP50-95(B)': last.get('metrics/mAP50-95(B)', float('nan')),
            'mAP50(B)': last.get('metrics/mAP50(B)', float('nan')),
            'best_pt': str(best) if best.exists() else 'MISSING'
        })
    return pd.DataFrame(rows).sort_values('mAP50-95(P)', ascending=False)

runs = [
    "/home/student/surg-pose-sim2real/runs_pose/synth_v8m_960",
    "/home/student/surg-pose-sim2real/runs_pose/synth_v8s_896",
    "/home/student/surg-pose-sim2real/runs_pose/synth_v8n_896",
]
summary = summarize_runs(runs)
summary


,run,model,imgsz,epochs,mAP50-95(P),mAP50(P),P(P),R(P),mAP50-95(B),mAP50(B),best_pt
0,synth_v8m_960,yolov8m-pose.pt,960,200,0.75221,0.94049,0.94886,0.93206,0.96342,0.99008,/home/student/surg-pose-sim2real/runs_pose/syn...
1,synth_v8s_896,yolov8s-pose.pt,896,200,0.71683,0.87720,0.90497,0.88200,0.95068,0.99413,/home/student/surg-pose-sim2real/runs_pose/syn...
2,synth_v8n_896,yolov8n-pose.pt,896,150,0.40926,0.83920,0.84428,0.84788,0.94232,0.98639,/home/student/surg-pose-sim2real/runs_pose/syn...


In [2]:
BEST = "/home/student/surg-pose-sim2real/runs_pose/synth_v8m_960/weights/best.pt"

In [ ]:
!python -m src.video \
  --source /datashare/project/vids_test/4_2_24_A_1.mp4 \
  --out /home/student/surg-pose-sim2real/outputs/results_synthetic_only.mp4 \
  --weights /home/student/surg-pose-sim2real/runs_pose/synth_v8m_960/weights/best.pt \
  --imgsz 960 --conf 0.25 --iou 0.5 --device 0 --stride 2


## Phase 3: Model Refinement on Unlabeled Real Data

In [31]:
!cd /home/student/surg-pose-sim2real;\
python -m src.refine_unsupervised \
  --teacher runs_pose/synth_v8m_960/weights/best.pt \
  --video /datashare/project/vids_test/4_2_24_A_1.mp4 \
  --synthetic_yaml outputs/datasets/surg.yaml \
  --out_root outputs/pseudo_real_v1 \
  --imgsz 960 \
  --det_conf 0.40 --kp_conf 0.50 --min_kps 3 \
  --stride 2 --max_frames 1150 \
  --train_epochs 25 --batch 16 --lr0 0.001 --device 0 --run_name refine_v1


[refine] extracted 1150 frames -> outputs/pseudo_real_v1/images
[refine] kept 43 images with ≥1 pseudo-label (label files exist for all frames)
[refine] wrote combined YAML -> outputs/pseudo_real_v1/combined.yaml
New https://pypi.org/project/ultralytics/8.3.190 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.184 🚀 Python-3.9.23 torch-2.0.0+cu118 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=outputs/pseudo_real_v1/combined.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=

## Extracte frames

In [32]:
!python /home/student/surg-pose-sim2real/src/uda_extract_frames.py \
  --videos /datashare/project/vids_test \
  --out-images /home/student/surg-pose-sim2real/outputs/pseudo/images/train \
  --fps 5 --ext jpg

4_2_24_A_1.mp4: saved 900 frames to /home/student/surg-pose-sim2real/outputs/pseudo/images/train
4_2_24_A_1_small.mp4: saved 900 frames to /home/student/surg-pose-sim2real/outputs/pseudo/images/train
done.


## Pseudo labels generating

In [33]:
!python /home/student/surg-pose-sim2real/src/uda_pseudo_label_pose.py \
  --images /home/student/surg-pose-sim2real/outputs/pseudo/images/train \
  --labels-out /home/student/surg-pose-sim2real/outputs/pseudo/labels/train \
  --model /home/student/surg-pose-sim2real/runs_pose/synth_v8m_960/weights/best.pt \
  --imgsz 960 --conf 0.6 --kp-conf 0.6 --oks-nms 0.85 --kpts 5 --device 0

Found 1800 images.
50/1800 frames processed
100/1800 frames processed
150/1800 frames processed
200/1800 frames processed
250/1800 frames processed
300/1800 frames processed
350/1800 frames processed
400/1800 frames processed
450/1800 frames processed
500/1800 frames processed
550/1800 frames processed
600/1800 frames processed
650/1800 frames processed
700/1800 frames processed
750/1800 frames processed
800/1800 frames processed
850/1800 frames processed
900/1800 frames processed
950/1800 frames processed
1000/1800 frames processed
1050/1800 frames processed
1100/1800 frames processed
1150/1800 frames processed
1200/1800 frames processed
1250/1800 frames processed
1300/1800 frames processed
1350/1800 frames processed
1400/1800 frames processed
1450/1800 frames processed
1500/1800 frames processed
1550/1800 frames processed
1600/1800 frames processed
1650/1800 frames processed
1700/1800 frames processed
1750/1800 frames processed
1800/1800 frames processed
pseudo-labels saved to: /home

## Temporal consistency filter (remove one-off/noisy frames)

In [34]:
!python /home/student/surg-pose-sim2real/src/uda_temporal_filter.py \
  --images  /home/student/surg-pose-sim2real/outputs/pseudo/images/train \
  --labels  /home/student/surg-pose-sim2real/outputs/pseudo/labels/train \
  --min-track 3 --iou-thr 0.5 --oks-thr 0.6 --kpts 5


Temporal filter done. kept detections with tracks ≥ 3


# Build combined YAML (synthetic + pseudo-real)

In [35]:
!python /home/student/surg-pose-sim2real/src/uda_make_yaml.py \
  --synthetic-root /home/student/surg-pose-sim2real/outputs/datasets  \
  --pseudo-root    /home/student/surg-pose-sim2real/outputs/pseudo \
  --out-yaml       /home/student/surg-pose-sim2real/outputs/train_pose_uda.yaml \
  --names          "needle_holder,tweezers" \
  --kpts 5

Wrote: /home/student/surg-pose-sim2real/outputs/train_pose_uda.yaml
path: /home/student/surg-pose-sim2real/outputs
train:
- /home/student/surg-pose-sim2real/outputs/datasets/images/train
- /home/student/surg-pose-sim2real/outputs/pseudo/images/train
val: /home/student/surg-pose-sim2real/outputs/datasets/images/val
names:
- needle_holder
- tweezers
kpt_shape:
- 5
- 3



## Fine-tune from teacher weights (domain adaptation)

In [43]:
from ultralytics import YOLO


BEST = "/home/student/surg-pose-sim2real/runs_pose/synth_v8m_960/weights/best.pt"
DATA = "/home/student/surg-pose-sim2real/outputs/pseudo_real_v1/combined.yaml"

model = YOLO(BEST)
model.train(
    data=DATA,
    imgsz=960,
    epochs=25,
    batch=16,
    device=0,
    lr0=0.001,
    cos_lr=True,
    close_mosaic=10,
    freeze=10,     # you can set 0–5 if you want to adapt more layers
    cache=True,
    project="/home/student/surg-pose-sim2real/runs_pose",
    name="refine_v1_yolo8m_960",
)


New https://pypi.org/project/ultralytics/8.3.190 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.184 🚀 Python-3.9.23 torch-2.0.0+cu118 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/student/surg-pose-sim2real/outputs/pseudo_real_v1/combined.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/student/surg-pose-sim2real/runs_pose/synth_v8m_960/weights/best

train: Scanning /home/student/surg-pose-sim2real/outputs/datasets/labels/train... 2185 images, 1122 backgrounds, 0 corrupt: 100%|██████████| 2185/2185 [00:02<00:00, 938.56it/s] 

train: New cache created: /home/student/surg-pose-sim2real/outputs/datasets/labels/train.cache



libpng warning: iCCP: known incorrect sRGB profile


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        115        113      0.975      0.972      0.986      0.966      0.941      0.932      0.951      0.798



25 epochs completed in 0.235 hours.
Optimizer stripped from /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960/weights/last.pt, 53.2MB
Optimizer stripped from /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960/weights/best.pt, 53.2MB

Validating /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960/weights/best.pt...
Ultralytics 8.3.184 🚀 Python-3.9.23 torch-2.0.0+cu118 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
YOLOv8m-pose summary (fused): 101 layers, 26,403,283 parameters, 0 gradients, 80.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.38it/s]


                   all        115        113      0.975      0.972      0.986      0.966      0.941      0.932      0.951      0.798
         needle_holder         62         62      0.952      0.984      0.986       0.97      0.883      0.903      0.916      0.756
              tweezers         51         51      0.997      0.961      0.986      0.962      0.999      0.961      0.986       0.84
Speed: 1.3ms preprocess, 4.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2cec527970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.03403

## Inference video

In [44]:
!python /home/student/surg-pose-sim2real/src/video.py \
  --source  /datashare/project/vids_test/4_2_24_A_1.mp4 \
  --out     /home/student/surg-pose-sim2real/outputs/results_refined.mp4 \
  --weights /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960/weights/best.pt \
  --imgsz 960 --conf 0.25 --iou 0.5 --device 0 --stride 2


[video] Wrote: /home/student/surg-pose-sim2real/outputs/results_refined.mp4


## Compare Runs

In [46]:
!python /home/student/surg-pose-sim2real/src/compare_runs.py \
  --base /home/student/surg-pose-sim2real/runs_pose/synth_v8m_960 \
  --ref  /home/student/surg-pose-sim2real/runs_pose/refine_v1_yolo8m_960 \
  --out  /home/student/surg-pose-sim2real/outputs/analysis/refine_compare \
  --video /datashare/project/vids_test/4_2_24_A_1.mp4 \
  --imgsz 960 --conf 0.25 --iou 0.5 --stride 10 --max-frames 120 --device 0


[compare] Wrote qualitative video: /home/student/surg-pose-sim2real/outputs/analysis/refine_compare/qualitative/qual_compare.mp4
[compare] Done. Artifacts in: /home/student/surg-pose-sim2real/outputs/analysis/refine_compare


## Upload to GitHub

In [47]:
!cd /home/student/surg-pose-sim2real
!git status  # see if this is already a git repo


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   predict.py
	modified:   refine_unsupervised.py
	modified:   synthetic_data_generator.py
	modified:   train.py
	modified:   video.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../assets/
	../docs/phase1_stats.py
	../docs/phase1_vis.py
	../runs_pose/
	Main.ipynb
	compare_runs.py
	main_computer_vision_project.ipynb
	uda_extract_frames.py
	uda_make_yaml.py
	uda_pseudo_label_pose.py
	uda_temporal_filter.py

no changes added to commit (use "git add" and/or "git commit -a")
